## Introduction
In this example, we show that how gear can be applied to find the bounds of error of a digital signal processing circuit throught the fixed-number lengths of the internal calculation.

The errors of computation of a digital signal processing circuit come from three sources: truncation error, quantization error, and inaccurate source.
1.  Truncation Error:
The truncation error is the error when a fixed-point number is converted from a longer representation to a shorter representation, which means the bits are not sufficient to represent the number, and thus the number must be truncated.
The 
2. Quantization Error:
The quantization error occurs when the an analog signal is converted into a digital signal, or an algorithm requires real numbers as its coefficients. As the fixed number cannot represent any real number with infinite precision. The number has to be quantized and thus are not as the exact value as the analog signal.
3. Inaccurate Source:
This error comes from the measurement in the analog signal because of noise and non-ideal effect on the sensor, and could be seen as a random noise on the analog signal.

The determination of the number of bits for fixed-point number requires tradeoff between the accuracy and the performance of the circuit.
Power, timing, and area of the circuit can benefit from reducing the number of bits for representation of fixed-point number as the gate count can be reduced, which results in shorter critical path length for optimization and smaller core footprint.
However, the accuracy of the signal processing decreases as more error is introduced in the computation and propagated to the output of the circuit.

Therefore, in digital signal processing circuit design. Exploration of the word lengths for all fixed-point number is a crucial stage from algorithm design to circuit implementation for optimizing the circuit performance while satisfying the system requirements.


## Preliminaries
#### Representation of Fixed-point Number

A word length of a fixed-point number can be described as a tuple $(n, p)$, where $n$ denotes the number of bits, and $p$ is the position of the binary point, with 0 position means the position before the most significant bit.

For example, A fixed-point number with word length $(5,2)$ with bit value "11011" means the binary number $11.011$, which is equivalent to $3.375$ in decimal.

In [1]:
from gear.terms.polyhedra.loaders import readContract, writeContract

### Forming contract for capturing error of fixed-point number

Here we introduce how to form the contract for the fixed-point number operation.

##### General operation
TODO
##### Addition
TODO
##### Multiplication
TODO

In the following, we import the tools for calculating the value for each operation given the word length of the input and output.

In [2]:
from tool import *

We use the class PortWordLength to represent the word length.
It also contains the known error of the number for propagating the error.

#### Example 1
Consider the following system with only two adder.
TODO: put figure

The following function is used to generate contracts, considering all error types.

In [3]:
def form_contract(in_port1, in_port2, out_port, operation):
    ret_contract = {}
    ret_contract["InputVars"] = [f"{in_port1.name}_a", f"{in_port1.name}_e",
                                 f"{in_port2.name}_a", f"{in_port2.name}_e"]
    ret_contract["OutputVars"] = [f"{out_port.name}_a", f"{out_port.name}_e"]
    a_bound = get_assumption_bound(in_port_1=in_port1, in_port_2=in_port2, out_port=out_port, operation_length_fn=operation)
    ret_contract["assumptions"] =  [{"coefficients":{f"{in_port1.name}_a":1, f"{in_port2.name}_a":1},
                                   "constant":a_bound}]
    if operation == "add":
        error_bound = get_guarantee_bound(in_port_1=in_port1, in_port_2=in_port2, out_port=out_port, operation_length_fn=operation)
        actual_val_bound = get_actual_possible_value(in_port=out_port)
        ret_contract["guarantees"] =  [{"coefficients":{f"{in_port1.name}_e":-1, f"{in_port2.name}_e":-1, f"{out_port.name}_e": 1},
                                    "constant":error_bound},
                                    {"coefficients":{f"{out_port.name}_a": 1}, "constant":actual_val_bound},
                                    ]
    elif operation == "mult":
        pass

    return ret_contract

In [4]:
    contracts = []
    p1 = PortWordLength(n=5, p=2, name = "p1")
    p2 = PortWordLength(n=5, p=3, name = "p2")
    p3 = PortWordLength(n=5, p=3, name = "p3")
    c1 = form_contract(in_port1=p1, in_port2=p2, out_port=p3, operation="add")
    contracts.append(c1)
    p4 = PortWordLength(n=7, p=3, name = "p4")
    p5 = PortWordLength(n=6, p=3, name = "p5")
    c2 = form_contract(in_port1=p3, in_port2=p4, out_port=p5, operation="add")

Find add length
in1:  Port: p1, (n, p) = (5, 2), e = None, a = None
in2:  Port: p2, (n, p) = (5, 3), e = None, a = None
(n*, p*) = (7, 4)
Find add length
in1:  Port: p1, (n, p) = (5, 2), e = None, a = None
in2:  Port: p2, (n, p) = (5, 3), e = None, a = None
(n*, p*) = (7, 4)
Theoretical out:  Port: , (n, p) = (7, 4), e = None, a = None
Find add length
in1:  Port: p3, (n, p) = (5, 3), e = None, a = None
in2:  Port: p4, (n, p) = (7, 3), e = None, a = None
(n*, p*) = (8, 4)
Find add length
in1:  Port: p3, (n, p) = (5, 3), e = None, a = None
in2:  Port: p4, (n, p) = (7, 3), e = None, a = None
(n*, p*) = (8, 4)
Theoretical out:  Port: , (n, p) = (8, 4), e = None, a = None


In [5]:
contract1 = readContract(c1)
contract2 = readContract(c2)


In [6]:
print("Contract 1:\n" + str(contract1))
print("Contract 2:\n" + str(contract2))

Contract 1:
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>]
OutVars:[<Var p3_a>, <Var p3_e>]
A: 1*p1_a + 1*p2_a <= 8
G: -1*p1_e + -1*p2_e + 1*p3_e <= 0.125, 1*p3_a <= 7.75
Contract 2:
InVars: [<Var p3_a>, <Var p3_e>, <Var p4_a>, <Var p4_e>]
OutVars:[<Var p5_a>, <Var p5_e>]
A: 1*p3_a + 1*p4_a <= 8
G: -1*p3_e + -1*p4_e + 1*p5_e <= 0.0625, 1*p5_a <= 7.875


In [7]:
contract_sys = contract1.compose(contract2)
print("Contract Sys:\n" + str(contract_sys))

Contract Sys:
InVars: [<Var p1_a>, <Var p1_e>, <Var p2_a>, <Var p2_e>, <Var p4_a>, <Var p4_e>]
OutVars:[<Var p5_a>, <Var p5_e>]
A: 1*p4_a <= 0.250000000000000, 1*p1_a + 1*p2_a <= 8
G: -1*p1_e + -1*p2_e + -1*p4_e + 1*p5_e <= 0.187500000000000, 1*p5_a <= 7.875
